In [3]:
import argparse
import os
import random

import torch
import torch.nn as nn

import tarfile

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

import torchvision
import torchvision.transforms as transforms
import torchvision.utils as utils

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import time

try:
    torch.use_deterministic_algorithms(True)
    torch.backends.cudnn.deterministic = True
except:
    pass

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# If you encounter some issues regarding cuda device, e.g., "RuntimeError: CUDA Out of memory error",
# try to switch the device to cpu by using the following code

# device = torch.device('cpu')

print('Device:', device)


!pip install onedrivedownloader



Device: cuda:0
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
with tarfile.open('./train.tar', 'r') as td:
    td.extractall('./miniImagenet')

In [4]:
# Download and preprocess MiniImageNet dataset
transform = transforms.Compose([
   # transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Adjust the path to your MiniImageNet dataset directory
miniimagenet_path = './miniImagenet/train'
dataset = datasets.ImageFolder(miniimagenet_path, transform=transform)


In [ ]:

# Split the dataset

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

print('total number of total set: {}'.format(len(dataset)))
print('total number of training set: {}'.format(len(train_set)))
print('total number of val set: {}'.format(len(val_set)))
print('total number of test set: {}'.format(len(test_set)))

# Create data loaders
train_loader = DataLoader(train_set, batch_size=64, shuffle=False)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [5]:
from sklearn.model_selection import train_test_split


train_set, test_set = train_test_split(dataset, test_size=0.2)
val_set, test_set = train_test_split(test_set, test_size=0.5)

print('total number of total set: {}'.format(len(dataset)))
print('total number of training set: {}'.format(len(train_set)))
print('total number of test set: {}'.format(len(test_set)))
print('total number of val set: {}'.format(len(val_set)))

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
valid_loader = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)


total number of total set: 38400
total number of training set: 30720
total number of test set: 3840
total number of val set: 3840


In [4]:
# Create data loaders
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)


In [6]:
num_classes = len(dataset.classes)
net = torchvision.models.resnet18(pretrained=False)
net.fc = nn.Linear(net.fc.in_features, num_classes)

In [18]:
# training function

# evaluation function
def eval(net, data_loader):
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        net = net.cuda()
    net.eval()
    correct = 0.0
    num_images = 0.0
    for i_batch, (images, labels) in enumerate(data_loader):
        if use_cuda:
            images = images.cuda()
            labels = labels.cuda()
        outs = net(images) 
#         _, preds = outs.max(1)
        preds = outs.argmax(dim=1)
        correct += preds.eq(labels).sum()
        num_images += len(labels)

    acc = correct / num_images
    return acc

def train(net, train_loader, valid_loader):
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01)

    use_cuda = torch.cuda.is_available()
    if use_cuda:
        net = net.cuda()
        
    epoches = 15
    for epoch in range(epoches):
        net.train()
        correct = 0.0 # used to accumulate number of correctly recognized images
        num_images = 0.0 # used to accumulate number of images
        for i_batch, (images, labels) in enumerate(train_loader):
            if use_cuda:
                images = images.cuda()
                labels = labels.cuda()
            optimizer.zero_grad()
            outs = net(images) 
            loss = loss_function(outs, labels)
            loss.backward()
            optimizer.step()
            preds = outs.argmax(dim=1)
            correct += preds.eq(labels).sum().item()
            num_images += len(labels) 
        
        acc = correct / num_images
        acc_eval = eval(net, valid_loader)
        print('epoch: %d, lr: %f, accuracy: %f, loss: %f, valid accuracy: %f' % (epoch, optimizer.param_groups[0]['lr'], acc, loss.item(), acc_eval))

    return net



In [19]:
train_net = train(net, train_loader, valid_loader)

epoch: 0, lr: 0.010000, accuracy: 0.398210, loss: 2.514230, valid accuracy: 0.305990
epoch: 1, lr: 0.010000, accuracy: 0.443457, loss: 1.794717, valid accuracy: 0.292448
epoch: 2, lr: 0.010000, accuracy: 0.490853, loss: 1.938201, valid accuracy: 0.297396
epoch: 3, lr: 0.010000, accuracy: 0.546322, loss: 1.801848, valid accuracy: 0.299479
epoch: 4, lr: 0.010000, accuracy: 0.600846, loss: 1.499800, valid accuracy: 0.322656
epoch: 5, lr: 0.010000, accuracy: 0.672493, loss: 1.356383, valid accuracy: 0.316146
epoch: 6, lr: 0.010000, accuracy: 0.747591, loss: 0.980958, valid accuracy: 0.334375
epoch: 7, lr: 0.010000, accuracy: 0.826628, loss: 0.691682, valid accuracy: 0.334115
epoch: 8, lr: 0.010000, accuracy: 0.891927, loss: 0.485882, valid accuracy: 0.304427
epoch: 9, lr: 0.010000, accuracy: 0.945508, loss: 0.336064, valid accuracy: 0.345573
epoch: 10, lr: 0.010000, accuracy: 0.977344, loss: 0.223478, valid accuracy: 0.334896
epoch: 11, lr: 0.010000, accuracy: 0.990299, loss: 0.099559, val

In [12]:
torch.use_deterministic_algorithms(False)